In [1]:
import os
import requests
import time
import random 
from tqdm import tqdm 
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import uuid
import json

In [ ]:
import os
import time
import json
import uuid
import requests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- HÀM HỖ TRỢ ---
def download_image(img_url, save_folder):
    """Tải ảnh và trả về tên file local"""
    try: 
        # Thêm User-Agent để tránh bị server chặn request ảnh
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(img_url, headers=headers, stream=True, timeout=10)
        
        if response.status_code == 200:
            # Xử lý đuôi file
            ext = img_url.split('.')[-1].split('?')[0]
            if len(ext) > 4 or not ext: ext = "jpg"
            
            filename = f"{uuid.uuid4()}.{ext}"
            file_path = os.path.join(save_folder, filename)
            
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            return filename
    except Exception as e:
        # Chỉ in lỗi ngắn gọn để không làm rối màn hình
        pass 
    return None

# --- SETUP ---
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new') # Chạy ẩn
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=chrome_options)

# Cấu hình đường dẫn
root_dir = './sict_corpus/tintuc'
images_dir = os.path.join(root_dir, 'images')
os.makedirs(root_dir, exist_ok=True)
os.makedirs(images_dir, exist_ok=True)

n_pages = 50
news_id = 0

# Sử dụng Explicit Wait
wait = WebDriverWait(driver, 10)

try:
    for page_idx in tqdm(range(1, n_pages + 1), desc="Pages"):
        try:
            # 1. Vào trang danh sách
            main_url = f'https://sict.haui.edu.vn/vn/tin-tuc/{page_idx}'
            driver.get(main_url)
            
            # Lấy danh sách link (Dùng wait để đảm bảo list đã load)
            news_lst_xpath = '//section[contains(@class, "irs-blog-field")]//h2/a'
            wait.until(EC.presence_of_element_located((By.XPATH, news_lst_xpath)))
            
            news_tags = driver.find_elements(By.XPATH, news_lst_xpath)
            # Lưu lại list URL để tránh lỗi StaleElement khi chuyển trang
            news_page_urls = [tag.get_attribute('href') for tag in news_tags]

            # 2. Vào từng bài viết
            for news_page_url in news_page_urls:
                try:
                    driver.get(news_page_url)
                    
                    # Định vị Main Content
                    main_content_xpath = '//section[contains(@class, "irs-blog-field")]//div[@class="col-md-8"]'
                    try:
                        main_content_tag = wait.until(EC.presence_of_element_located((By.XPATH, main_content_xpath)))
                    except:
                        # Nếu không load được content, bỏ qua bài này
                        continue

                    # --- TRÍCH XUẤT DỮ LIỆU ---
                    
                    # Title
                    title = ""
                    try:
                        title = main_content_tag.find_element(By.XPATH, './/p[@class="pTitle"]').text.strip()
                    except: pass

                    # Abstract
                    abstract = ""
                    try:
                        abstract = main_content_tag.find_element(By.XPATH, './/p[@class="pHead"]').text.strip()
                    except: pass

                    # Content & Images
                    content_text = ""
                    images_data = []
                    
                    try:
                        # Lấy tất cả thẻ pBody
                        paragraphs_tags = main_content_tag.find_elements(By.XPATH, './/p[@class="pBody"]')
                        text_parts = []
                        
                        # Logic: Bỏ thẻ cuối cùng ([:-1])
                        target_paragraphs = paragraphs_tags[:-1] if paragraphs_tags else []

                        for p_tag in target_paragraphs:
                            # 1. Lấy Text
                            text = p_tag.text.strip()
                            if text:
                                text_parts.append(text)
                            
                            # 2. Lấy Ảnh (QUAN TRỌNG: Chỉ tìm trong p_tag hiện tại)
                            # Code cũ của bạn dùng main_content_tag ở đây gây lỗi lặp dữ liệu
                            imgs_in_p = p_tag.find_elements(By.TAG_NAME, "img")
                            
                            for img in imgs_in_p:
                                src = img.get_attribute('src')
                                if src:
                                    saved_filename = download_image(src, images_dir)
                                    if saved_filename:
                                        images_data.append({
                                            "original_url": src,
                                            "local_filename": saved_filename,
                                            "relative_path": f"images/{saved_filename}"
                                        })

                        content_text = '\n'.join(text_parts)

                    except Exception as e:
                        print(f"Lỗi parse body: {e}")

                    # --- LƯU JSON ---
                    article_data = {
                        "id": f"sict_{news_id:05d}",
                        "url": news_page_url,
                        "title": title,
                        "abstract": abstract,
                        "content": content_text,
                        "images": images_data 
                    }

                    news_filename = f"new_sict_{news_id:05d}.json"
                    news_savepath = os.path.join(root_dir, news_filename)
                    
                    with open(news_savepath, 'w', encoding='utf-8') as f:
                        json.dump(article_data, f, ensure_ascii=False, indent=4)
                    
                    news_id += 1
                    
                    # Không cần driver.back() vì vòng lặp sẽ gọi driver.get() trang mới ngay

                except Exception as inner_e:
                    print(f"Lỗi bài viết {news_page_url}: {inner_e}")

        except Exception as e:
            print(f"Lỗi trang danh sách {page_idx}: {e}")

finally:
    # Đảm bảo tắt trình duyệt khi xong hoặc gặp lỗi
    driver.quit()
    print("Hoàn tất crawl.")

  2%|▏         | 1/50 [04:10<3:24:23, 250.27s/it]


KeyboardInterrupt: 

In [14]:
len(news_page_urls)

8